# Анализ NPS клиентов телекоммуникационной компании

### Постановка задачи

Заказчик исследования — телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности среди клиентов из России. 

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».
Компания провела опрос и попросила подготовить дашборд с его итогами.

Чтобы оценить результаты опроса, оценки делят на три группы:
* 9-10 баллов — «cторонники» (англ. promoters);
* 7-8 баллов — «нейтралы» (англ. passives);
* 0-6 баллов — «критики» (англ. detractors).

### Описание данных

Таблица `users`:

* `user_id` — Идентификатор клиента, первичный ключ таблицы
* `lt_day` — Количество дней «жизни» клиента
* `age` — Возраст клиента в годах
* `gender_segment` — Пол клиента (1 – женщина, 0 – мужчина)
* `os_name` — Тип операционной системы
* `cpe_type_name` — Тип устройства
* `location_id` — Идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location
* `age_gr_id` — Идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment
* `tr_gr_id` — Идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице traffic_segment
* `lt_gr_id` — Идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице lifetime_segment
* `nps_score` — Оценка клиента в NPS-опросе (от 1 до 10)

Таблица `location`:

* `location_id` — Идентификатор записи, первичный ключ
* `country` — Страна
* `city` — Город

Таблица `age_segment`:

* `age_gr_id`— Идентификатор сегмента, первичный ключ
* `bucket_min` — Минимальная граница сегмента
* `bucket_max` — Максимальная граница сегмента
* `title` — Название сегмента

Таблица `traffic_segment`:

* `tr_gr_id` — Идентификатор сегмента, первичный ключ
* `bucket_min` — Минимальная граница сегмента
* `bucket_max` — Максимальная граница сегмента
* `title` — Название сегмента

Таблица `lifetime_segment`:

* `lt_gr_id` — Идентификатор сегмента, первичный ключ
* `bucket_min` — Минимальная граница сегмента
* `bucket_max` — Максимальная граница сегмента
* `title` — Название сегмента



### План исследования

* Импортирование библиотек
* Подключение к базе данных
* Выгрузка данных с помощью SQL-запроса
* Создание дашборда в Tableau
* Сделать презентацию с основными выводами

### Импортирование библиотек

In [1]:
# Импортирование библиотек
import os
import pandas as pd
from sqlalchemy import create_engine

### Подключение к базе данных

In [2]:
# путь к БД на вашем компьютере (например, в той же папке, что и тетрадь)
path_to_db_local = '/Users/aaleksakhin/Data/Data Bases/telecomm_csi.db'
# путь к БД на платформе
path_to_db_platform = '/datasets/telecomm_csi.db'
# итоговый путь к БД
path_to_db = None

# если путь на вашем компьютере ведёт к БД, то он становится итоговым
if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
# иначе: если путь на платформе ведёт к БД, то он становится итоговым
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
# иначе выводится сообщение о том, что файл не найден
else:
    raise Exception('Файл с базой данных SQLite не найден!')

# если итоговый путь не пустой
if path_to_db:
    # то создаём подключение к базе
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)


### Выгрузка данных

Выгрузим следующие данные для построения дашборда

* `user_id` — Идентификатор клиента, первичный ключ таблицы
* `lt_day` — Количество дней «жизни» клиента
* `is_new` — Поле хранит информацию о том, является ли клиент новым
* `age` — Возраст клиента в годах
* `gender_segment` — Пол клиента (1 – женщина, 0 – мужчина)
* `os_name` — Тип операционной системы
* `cpe_type_name` — Тип устройства
* `country` — Страна
* `city` — Город
* `age_segment` — Возрастной сегмент
* `traffic_segment` — Сегмент по объёму потребляемого трафика
* `lifetime_segment`— Сегмент по количеству дней «жизни»
* `nps_score` — Оценка клиента в NPS-опросе
* `nps_group` — Поле хранит информацию о том, к какой группе относится оценка клиента в опросе	

Определить, является ли клиент новым, можно по количеству дней «жизни». Клиента считают новым, если количество дней «жизни» составляет не более 365 дней. 



In [3]:
# SQL-запрос
query = """

    SELECT u.user_id,
           u.lt_day,
           CASE
                   WHEN u.lt_day <= 365 THEN 'True'
                   WHEN u.lt_day > 365 THEN 'False'
               END AS is_new,
               u.age,
               u.gender_segment,
               u.os_name,
               u.cpe_type_name,
               l.country,
               l.city,
               t.title as traffic_segment,
               lt.title as lifetime_segment,
               a.title as age_segment,
               u.nps_score,
               CASE 
                   WHEN u.nps_score > 0 and u.nps_score <= 6 THEN 'Detractors'
                   WHEN u.nps_score >= 7 and u.nps_score <= 8 THEN 'Passives' 
                   ELSE 'Promoters'
               END AS nps_group
    FROM user AS u
    LEFT JOIN location AS l ON u.location_id=l.location_id
    LEFT JOIN age_segment AS a ON u.age_gr_id=a.age_gr_id
    LEFT JOIN traffic_segment AS t ON u.tr_gr_id=t.tr_gr_id
    LEFT JOIN lifetime_segment AS lt ON u.lt_gr_id=lt.lt_gr_id
    
    """

In [4]:
# создаём датафрейм по данным запроса
df = pd.read_sql(query, engine)

# проверим выгрузку
df.head(3)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,traffic_segment,lifetime_segment,age_segment,nps_score,nps_group
0,A001A2,2320,False,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,04 1-5,08 36+,05 45-54,10,Promoters
1,A001WF,2344,False,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,04 1-5,08 36+,05 45-54,10,Promoters
2,A003Q7,467,False,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,08 20-25,06 13-24,06 55-64,10,Promoters


### Ссылки на дашборд и презентацию

Ссылка на дашборд - https://public.tableau.com/views/nps_project/1?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

Ссылка на презентацию - https://drive.google.com/file/d/1APB-suqL98g9wdb1FW5zhZVIx7LV8dw5/view?usp=sharing